In [1]:
import numpy as np
import torch
import sklearn
from pol.datasets.linear_regression import create_nd_dataset

In [2]:
theta = 0.839
dataset = create_nd_dataset(dim=20, num_sample=20, seed=1234)
X, Y = dataset[:]
print(f'X shape: {X.shape}, Y shape: {Y.shape}')

tensor([ 1.9068, -0.4792,  1.6930, -0.9532, -0.7236, -1.5276, -1.0329, -0.7259,
         1.8563, -0.9454, -0.2360,  0.4395,  1.4545,  1.4550,  0.6995,  0.6395,
         0.9430, -1.1090, -1.3117,  1.4817], dtype=torch.float64)
X shape: torch.Size([20, 20]), Y shape: torch.Size([20, 1])


In [3]:
def solve_lasso(X, Y, theta):
    from sklearn import linear_model
    print('lasso alpha={}'.format(theta))
    clf = linear_model.Lasso(
        alpha=theta / 2,
        fit_intercept=False,
        max_iter=100)
    clf.fit(X, Y)
    W = clf.coef_
    return W

In [4]:
def compute_loss(W):
    W = W.unsqueeze(-1) # Nx1
    l1 = (torch.matmul(X, W) - Y).square().squeeze(-1)
    l2 = theta * W.abs().squeeze(-1)
    return (l1 + l2).mean()

In [5]:
W_sk = solve_lasso(X.numpy(), Y.squeeze(-1).numpy(), theta=theta)
W_sk = torch.from_numpy(W_sk)
L_sk = compute_loss(W_sk)
print(f'W_sk: {W_sk}')
print(f'L_sk: {L_sk}')

lasso alpha=0.839
W_sk: tensor([ 1.0047, -0.0000,  0.5792, -0.0000, -0.0000, -1.1364, -0.5494, -0.0000,
         0.3137,  0.0000,  0.2430,  0.0000,  0.0610,  0.4075, -0.0000,  0.7827,
         0.5565,  0.0000, -0.0000,  1.6964])
L_sk: 3.734462022781372


In [6]:
W_pol = torch.tensor([-0.0003,-0.0004,0.0836,0.0195,0.0004,-0.0020,-0.0014,-0.0012,0.2181,-0.0004,-0.0101,-0.0000,-0.0421,0.0195,-1.6940,0.0024,-0.0006,-0.0005,0.0010,-0.0018])
L_pol = compute_loss(W_pol)
print(f'W_pol: {W_pol}')
print(f'L_pol: {L_pol}')

W_pol: tensor([-3.0000e-04, -4.0000e-04,  8.3600e-02,  1.9500e-02,  4.0000e-04,
        -2.0000e-03, -1.4000e-03, -1.2000e-03,  2.1810e-01, -4.0000e-04,
        -1.0100e-02, -0.0000e+00, -4.2100e-02,  1.9500e-02, -1.6940e+00,
         2.4000e-03, -6.0000e-04, -5.0000e-04,  1.0000e-03, -1.8000e-03])
L_pol: 22.54592514038086
